In [ ]:
import sys
sys.path.append('D:\ephys')

import numpy as np
import matplotlib.pyplot as plt
# %matplotlib notebook
import glob
import re # Regular expression operations
import pandas as pd
from scipy import signal
import mne
from tqdm import tqdm
import os
from os.path import exists
import pickle
import statsmodels.stats.api as sms

import utils
import ephys
import plotting
plt.rcParams["axes.labelsize"]=12
plt.rcParams["font.size"]=12
plt.rcParams["font.family"] = "Arial"

# # Create geometry distribution of electrode pads

In [ ]:
print(ephys.pad_to_array()[0])

# Load preprocessed data

In [ ]:
sessions = {
    'arena_0219':     '2021-02-19_mBWfus009_arena_ephys', 
    'ezm_0219':       '2021-02-19_mBWfus009_EZM_ephys', 
    'oft_0219':       '2021-02-19_mBWfus009_OF_ephys',
    
    'arena_0226':     '2021-02-26_mBWfus009_arena_ephys', 
    'ezm_0226':       '2021-02-26_mBWfus009_EZM_ephys', 
    'oft_0226':       '2021-02-26_mBWfus009_OF_ephys',
    
    'arena_0305_bef': '2021-03-05_mBWfus009_before_arena_ephys',
    'cage_0305':      '2021-03-05_mBWfus009_cage_arena_ephys',
    'arena_0305_aft': '2021-03-05_mBWfus009_after_arena_ephys', 
    'ezm_0305':       '2021-03-05_mBWfus009_EZM_ephys', 
    'oft_0305':       '2021-03-05_mBWfus009_OF_ephys',
    
    'arena_0325':     '2021-03-25_mBWfus009_arena_ephys',
    'epm_0325':       '2021-03-25_mBWfus009_EPM_ephys'
           }

In [ ]:
arena_0219   = ephys.load_data(sessions['arena_0219'])
ezm_0219     = ephys.load_data(sessions['ezm_0219'])
oft_0219     = ephys.load_data(sessions['oft_0219'])

# traj_oft_0219 = ephys.load_traj(sessions['oft_0219'])
# events_oft_0219 = ephys.load_events(sessions['oft_0219'])


# arena_0226   = ephys.load_data(sessions['arena_0226'])
# ezm_0226     = ephys.load_data(sessions['ezm_0226'])
# oft_0226     = ephys.load_data(sessions['oft_0226'])

# arena_0305_bef = ephys.load_data(sessions['arena_0305_bef'])
# cage_0305      = ephys.load_data(sessions['cage_0305'])
# arena_0305_aft = ephys.load_data(sessions['arena_0305_aft'])

# ezm_0305       = ephys.load_data(sessions['ezm_0305'])
# oft_0305       = ephys.load_data(sessions['oft_0305'])
#
# arena_0325      = ephys.load_data(sessions['arena_0325'])
# epm_0325        = ephys.load_data(sessions['epm_0325'])


In [ ]:
arena_0219.keys()

In [ ]:
oft_0219_events =ephys.load_events(sessions['oft_0219'])
oft_0219_traj = ephys.load_traj(sessions['oft_0219'])
oft_0219_riostats = ephys.load_riostats(sessions['oft_0219'])

In [ ]:
oft_0219_riostats

In [ ]:
## extract 500hz data
lfp_arena_0219 = ephys.get_lfp(arena_0219)
lfp_ezm_0219 = ephys.get_lfp(ezm_0219)
lfp_oft_0219 = ephys.get_lfp(oft_0219)

# lfp_arena_0226 = get_lfp(arena_0226)
# lfp_ezm_0226 = get_lfp(ezm_0226)
# lfp_oft_0226 = get_lfp(oft_0226)

# lfp_arena_0305_bef = get_lfp(arena_0305_bef)
# lfp_arena_0305_aft = get_lfp(arena_0305_aft)
# lfp_ezm_0305 = get_lfp(ezm_0305)
# lfp_oft_0305 = get_lfp(oft_0305)

## extract theta power
theta_power_arena_0219 = ephys.get_power(arena_0219, 'theta')
theta_power_ezm_0219 = ephys.get_power(ezm_0219, 'theta')
theta_power_oft_0219 = ephys.get_power(oft_0219, 'theta')

# theta_power_arena_0226 = get_theta_power(arena_0226)
# theta_power_ezm_0226 = get_theta_power(ezm_0226)
# theta_power_oft_0226 = get_theta_power(oft_0226)

# theta_power_arena_0305_bef = get_theta_power(arena_0305_bef)
# theta_power_arena_0305_aft = get_theta_power(arena_0305_aft)
# theta_power_ezm_0305 = get_theta_power(ezm_0305)
# theta_power_oft_0305 = get_theta_power(oft_0305)

# lfp_arena_0325 = get_lfp(arena_0325)
# lfp_epm_0325 = get_lfp(epm_0325)


In [ ]:
% -- Coherence stuff

In [ ]:
tpower_corr_matrix = np.zeros((len(lfp_mpfc),len(lfp_mpfc)))

for x_id, x in tqdm(enumerate(lfp_mpfc)):
    for y_id, y in enumerate(lfp_mpfc):
        coherence_mpfc[x_id,y_id] = coherence(x=x,y=y, fs=500)[1][:20].mean()